### Afazeres

- [ ] Juntar as etapas de limpeza das strings em uma só
- [ ] Juntar as partes das listas que _não_ contem "Reading" em uma só, que diz "Celebra-se hoje: {conteúdo}".
- [ ] Transformar as diferentes etapas em funções
    - As etapas são:
        1. Selecionar mês e ano
        2. Raspar o conteúdo do mês em questão
        3. Limpar o conteúdo do mês
        4. Juntar os santos celebrados em uma só _string_
- [ ] Criar um loop que raspe os dados de um ano inteiro
    - Neste loop, cada mês deve ser colocado em um arquivo .csv diferente


In [570]:
import requests, re
import pandas as pd
from bs4 import BeautifulSoup as bs4

In [3]:
MONTH=11
YEAR=2023

In [4]:
BASE_URL = "https://www.goarch.org/chapel/calendar?month={}&year={}&viewStyle=GridView&viewType=ViewReadings".format(MONTH, YEAR)

In [5]:
header = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36',
}

In [477]:
page = requests.get(BASE_URL, headers=header)
soup = bs4(page.content, "html.parser")
cal_day_list = soup.find_all(class_="cal-day")

regex_pattern = re.compile("\?date=\d{1,2}\/\d{1,2}\/\d{1,4}")
content_list = []
for index in range(len(cal_day_list)):
    data_list = []
    day = cal_day_list[index].find(class_="day")
    anchors = day.find_all('a')
    
    child = cal_day_list[index]
    if child.attrs['class'][0] == "out_of_range":
        continue
    
    for link in anchors:
        match = re.search(regex_pattern, str(link))
        if match != None:
            data_list.append(match.group().split('=')[1])

    event_list = child.find_all(class_="sub-event-right")
    reading_list = [item.find(class_="cal_link").text for item in event_list]
    data_list.append(reading_list)
    content_list.append(data_list)

In [535]:
for i in range(len(content_list)):
    for j in range(len(content_list[i][1])):
        content_list[i][1][j] = content_list[i][1][j].strip()

In [559]:
for i in range(len(content_list)):
    for j in range(len(content_list[i][1])):
        content_list[i][1][j] = content_list[i][1][j].replace("\n", " ")

In [564]:
for i in range(len(content_list)):
    for j in range(len(content_list[i][1])):
        content_list[i][1][j] = content_list[i][1][j].replace("  ", " ")

In [565]:
reading_dict = {}
for inlist in content_list:
    if len(inlist) > 1:
        reading_dict[inlist[0]] = inlist[1]

In [569]:
reading_dict

{'11/1/2023': ['David the Righteous of Evia',
  'Epistle Reading – I Corinthians 12:27-31;13:1-8',
  'Gospel Reading – Matthew 10:1, 5-8'],
 '11/2/2023': ['The Holy Martyrs Acindynos, Pegasios, Aphthonios, Elpidophoros, and Anempodistos',
  'Domnina, Domna, & Kyriaki, the Martyrs',
  'Epistle Reading – Colossians 4:2-9',
  'Gospel Reading – Luke 11:47-54; 12:1'],
 '11/3/2023': ['Acepsimas the Bishop, Joseph the Presbyter, & Aeithalas the Deacon, Martyrs of Persia',
  'Dedication of the Temple of the Holy Great Martyr George in Lydda',
  'George of Neapolis the New Martyr',
  'Holy Father and New Hieromartyr Gregory of Neapolis',
  'Theodore the Confessor, Bishop of Ancyra',
  'Winifred of Treffynnon',
  'Epistle Reading – Colossians 4:10-18',
  'Gospel Reading – Luke 12:2-12'],
 '11/4/2023': ['Bishop Raphael Hawaweeny of Brooklyn',
  'Joannicius the Great',
  'The Holy Hieromartyrs Nicander, Bishop of Myra, and Hermias the Presbyter',
  'Porphyrios the Mime',
  'Emperor John Batatze th

In [567]:
pd.DataFrame.from_dict(reading_dict, orient='index')

,0,1,2,3,4,5,6,7,8
11/1/2023,David the Righteous of Evia,Epistle Reading – I Corinthians 12:27-31;13:1-8,"Gospel Reading – Matthew 10:1, 5-8",None,None,None,None,None,None
11/2/2023,"The Holy Martyrs Acindynos, Pegasios, Aphthoni...","Domnina, Domna, & Kyriaki, the Martyrs",Epistle Reading – Colossians 4:2-9,Gospel Reading – Luke 11:47-54; 12:1,None,None,None,None,None
11/3/2023,"Acepsimas the Bishop, Joseph the Presbyter, & ...",Dedication of the Temple of the Holy Great Mar...,George of Neapolis the New Martyr,Holy Father and New Hieromartyr Gregory of Nea...,"Theodore the Confessor, Bishop of Ancyra",Winifred of Treffynnon,Epistle Reading – Colossians 4:10-18,Gospel Reading – Luke 12:2-12,None
11/4/2023,Bishop Raphael Hawaweeny of Brooklyn,Joannicius the Great,"The Holy Hieromartyrs Nicander, Bishop of Myra...",Porphyrios the Mime,Emperor John Batatze the Merciful,George Karslidis of Pontos,Epistle Reading – II Corinthians 5:1-10,Gospel Reading – Luke 9:1-6,None
11/5/2023,"Galaktion & his wife Episteme, the Martyrs of ...","Hermas, Patrobos, Gaios, Linos, & Philologos, ...",Matins Gospel Reading – John 21:14-25,Epistle Reading – Galatians 6:11-18,Gospel Reading – Luke 16:19-31,None,None,None,None
11/6/2023,Luke the Monk of Taormina,Epistle Reading – Hebrews 8:1-6,Gospel Reading – Luke 12:8-12,None,None,None,None,None,None
11/7/2023,33 Martyrs of Melitene,Lazarus the Wonderworker,Martyr Athenodorus,Alexander the Martyr of Thessaloniki,Epistle Reading – I Thessalonians 1:6-10,Gospel Reading – Luke 12:42-48,None,None,None
11/8/2023,Matins Gospel Reading – Matthew 18:10-20,Epistle Reading – Hebrews 2:2-10,Gospel Reading – Luke 10:16-21,None,None,None,None,None,None
11/9/2023,Onesiphorus and Porphyrius of Ephesus,"Matrona, Abbess of Constantinople",Theoktisti of the Isle of Lesbos,Symeon the Translator,Matins Gospel Reading – Matthew 24:42-47,Epistle Reading – Ephesians 5:8-19,"Gospel Reading – Matthew 4:25, 5:1-12",None,None
11/10/2023,"Erastus, Olympas, Rodion, Sosipater, Quartus, ...",Orestes the Martyr of Cappadocia,Holy Father Arsenius of Cappadocia,"Our Holy Father Gregory, Bishop of Assa",Epistle Reading – I Corinthians 4:9-16,Gospel Reading – Luke 13:31-35,None,None,None
